In [ ]:
from google.colab import files

uploaded = files.upload()



Saving j360_tenders.xlsx to j360_tenders (1).xlsx
Saving opportunities.xlsx to opportunities (1).xlsx


In [ ]:
import pandas as pd
from google.colab import files
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from collections import Counter


nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Text classification model** to make predictions on new opportunities

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import nltk
import string
from nltk.corpus import stopwords

opportunities = pd.read_excel('/content/opportunities.xlsx', engine='openpyxl')

# Split data into training and test sets
X = opportunities.drop('Summary', axis=1)
y = opportunities['Target Class']

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the 'Summary' column using TF-IDF vectorizer
X_tfidf = tfidf_vectorizer.fit_transform(opportunities['Summary'])

# Model Selection: Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Model Training
model.fit(X_tfidf, y)

# Load new opportunities
new_opportunities = pd.read_excel('/content/j360_tenders.xlsx', engine='openpyxl')

# Cleaning the data
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words and word.isalpha()]
    cleaned_text = ' '.join(words)
    return cleaned_text

new_opportunities['Summary'] = new_opportunities['Summary'].apply(clean_text)

# Remove duplicates
df = new_opportunities.drop_duplicates(subset='Summary', keep='first')
df = df.reset_index(drop=True)  # Use 'df' instead of 'new_opportunities' here

# Transform the 'Summary' column using the same TF-IDF vectorizer
X_new = tfidf_vectorizer.transform(df['Summary'])  # Use 'df' here

# Make predictions for new opportunities
new_predictions = model.predict(X_new)

# Calculate the total count of each unique prediction
prediction_counts = pd.Series(new_predictions).value_counts()

# Print the counts
print(prediction_counts)

# Add 'PredictedOutcome' to the DataFrame
df['PredictedOutcome'] = new_predictions

# Writing predictions to an Excel file
df.to_excel('predicted_opportunities.xlsx', index=False, engine='openpyxl')

print(new_predictions)


Other          107
Development     15
dtype: int64
['Development' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Development' 'Other' 'Other' 'Development' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Development' 'Other' 'Other'
 'Other' 'Development' 'Development' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Development' 'Development'
 'Development' 'Development' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Development'
 'Development' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Development' 'Other' 'Other' 'Other' 'Other'
 'Development' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Ot

**Linear regression using scikit-learn library** to predict deadlines based on

1.   Élément de liste:
2.   Élément de liste

location and the number of days until the deadline.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from datetime import datetime, timedelta

data = pd.read_excel('/content/j360_tenders.xlsx')

# Convert 'Location' to numerical labels using LabelEncoder
le = LabelEncoder()
data['Location'] = le.fit_transform(data['Location'])

# Convert 'Deadlines' to numerical values (e.g., days from a reference date)
reference_date = datetime(2023, 1, 1)  # Choose a reference date
data['DaysUntilDeadline'] = (pd.to_datetime(data['Deadlines'], format='%d/%m/%Y') - reference_date).dt.days

# Define features (X) and target (y)
X = data['Location'].values.reshape(-1, 1)
y = data['DaysUntilDeadline']

# Create a Linear Regression model
model = LinearRegression()

# Fit the model on the data
model.fit(X, y)


# predict a deadline for a specific location:
location_code = le.transform(['Kenya'])[0]
X_predict = [[location_code]]
predicted_days_until_deadline = model.predict(X_predict)

# Convert the predicted number of days back to a date
predicted_deadline = reference_date + timedelta(days=int(predicted_days_until_deadline[0]))
print(f"Predicted Deadline: {predicted_deadline.strftime('%d/%m/%Y')}")


Predicted Deadline: 17/02/2024




```
# Ce texte est au format code
```

**Linear Regression Model** that provides insights into how far in advance or behind deadlines are for different countries, making it useful for prioritizing tasks or assessing project timelines.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

data = pd.read_excel('/content/opportunities.xlsx')

# Convert 'Location' to numerical labels using LabelEncoder
le = LabelEncoder()
data['Country'] = le.fit_transform(data['Country'])
country_names = le.classes_  # Get the list of country names

# Convert 'Deadlines' to numerical values (e.g., days from a reference date)
reference_date = datetime(2023, 1, 1)  # Choose a reference date

# Adjust the format for parsing 'Deadline' data based on the actual formats in your data
def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%d %b %Y')
    except ValueError:
        pass

    try:
        return pd.to_datetime(date_str, format='%Y-%m-%d')
    except ValueError:
        pass

    return reference_date

data['DaysUntilDeadline'] = (parse_date(data['Deadline']) - reference_date)

# Define features (X) and target (y)
X = data['Country'].values.reshape(-1, 1)
y = data['DaysUntilDeadline'].dt.days  # Access the .dt attribute to get the days

# Linear Regression model
model = LinearRegression()

# Fit the model on the data
model.fit(X, y)

# Get the current date
current_date = datetime.now()

# Calculate the predicted deadlines for each country
data['PredictedDaysUntilDeadline'] = model.predict(X)

# Calculate the difference in days between the predicted deadline and the current date
data['DaysDifference'] = (current_date - (reference_date + pd.to_timedelta(data['PredictedDaysUntilDeadline'], unit='D'))).dt.days

# Calculate the average difference in days for each country
country_order = data.groupby('Country')['DaysDifference'].mean().reset_index()

# Sort the countries based on their average difference in days (reversed order)
country_order = country_order.sort_values(by='DaysDifference', ascending=False)

# Add the country names
country_order['CountryName'] = [country_names[i] for i in country_order['Country']]

print(country_order[['CountryName', 'DaysDifference']])


output_file = 'sorted_countries_deadlines.xlsx'
country_order[['CountryName', 'DaysDifference']].to_excel(output_file, index=False)
print(f"Data saved to {output_file}")


      CountryName  DaysDifference
0          Angola            -3.0
1        Cameroon           -10.0
2    Eastern Cape           -18.0
3          France           -25.0
4      Free State           -33.0
5          Gambia           -40.0
6         Gauteng           -48.0
7           Ghana           -55.0
8   KwaZulu-Natal           -63.0
9         Limpopo           -70.0
10        Morocco           -78.0
11     Mpumalanga           -85.0
12       National           -93.0
13     North West          -100.0
14  Northern Cape          -108.0
15   South Africa          -115.0
16         Uganda          -123.0
17   Western Cape          -130.0
Data saved to sorted_countries_deadlines.xlsx


The total keyword weight for every country

In [ ]:
data = pd.read_excel('/content/opportunities.xlsx')

# Group the data by 'Country' and calculate the total keyword weight for each country
country_grouped = data.groupby('Country')['Total_Keyword_Weight'].sum()

# Iterate through the grouped data and print each country and its total keyword weight
for country, total_weight in country_grouped.items():
    print(f'Country: {country}, Total Keyword Weight: {total_weight}')


Country: Angola, Total Keyword Weight: 8
Country: Cameroon, Total Keyword Weight: 0
Country: Eastern Cape, Total Keyword Weight: 66
Country: France, Total Keyword Weight: 6
Country: Free State, Total Keyword Weight: 5
Country: Gambia, Total Keyword Weight: 6
Country: Gauteng, Total Keyword Weight: 178
Country: Ghana, Total Keyword Weight: 8
Country: KwaZulu-Natal, Total Keyword Weight: 42
Country: Limpopo, Total Keyword Weight: 5
Country: Morocco, Total Keyword Weight: 14
Country: Mpumalanga, Total Keyword Weight: 43
Country: National, Total Keyword Weight: 49
Country: North West, Total Keyword Weight: 12
Country: Northern Cape, Total Keyword Weight: 6
Country: South Africa, Total Keyword Weight: 4
Country: Uganda, Total Keyword Weight: 8
Country: Western Cape, Total Keyword Weight: 47


**Random Forest Classifier** to predict opportunities based on the 'Country' and 'Total_Keyword_Weight' features. The 'OpportunityLabel' indicates whether an opportunity is considered 'better' (1) or not (0). Additionally, the feature importance of the model is provided in a separate Excel file.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

data = pd.read_excel('/content/opportunities.xlsx')

# Encode the 'Country' column
le = LabelEncoder()
data['Country'] = le.fit_transform(data['Country'])
country_names = le.classes_  # Get the list of country names

# the criteria for "better" opportunities
data['OpportunityLabel'] = data['Total_Keyword_Weight'].apply(lambda x: 1 if x >= 10 else 0)

# features and target
X = data[['Country', 'Total_Keyword_Weight']]
y = data['OpportunityLabel']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Add the 'OpportunityLabel' to the DataFrame
data['OpportunityLabel'] = y

# Decode the country labels back to their original names
data['Country'] = le.inverse_transform(data['Country'])

data.to_excel('/content/opportunities_with_opportunitylabel.xlsx', index=False)

# New data for prediction with multiple countries
new_data = pd.DataFrame({
    'Country': ['France', 'France', 'Uganda'],  # Replace with actual country names
    'Total_Keyword_Weight': [7, 37, 11]
})

# Encode the 'Country' column for the new data
new_data['Country'] = le.transform(new_data['Country'])

# Make predictions on the new data
predictions = model.predict(new_data[['Country', 'Total_Keyword_Weight']])  # Corrected this line

# Combine the predictions with the country names
new_data['Country'] = le.inverse_transform(new_data['Country'])  # Decode back to country names
result = pd.DataFrame({
    'Country': new_data['Country'],
    'Total_Keyword_Weight': new_data['Total_Keyword_Weight'],
    'OpportunityLabel': predictions
})

print("Predicted Opportunities:")
print(result)

# Display feature importance
importances = model.feature_importances_
feature_names = X.columns
feature_importance = pd.Series(importances, index=feature_names)
print("\nFeature Importance:")
print(feature_importance)

feature_importance.to_excel('/content/feature_importance.xlsx', header=['Importance'])


Accuracy: 1.0
Predicted Opportunities:
  Country  Total_Keyword_Weight  OpportunityLabel
0  France                     7                 0
1  France                    37                 1
2  Uganda                    11                 1

Feature Importance:
Country                 0.02549
Total_Keyword_Weight    0.97451
dtype: float64


**Random Forect Classifier** to predict the probability of each country being in the "better" category (sorting the opportunities: 1 for a good opportunity and 0 for not that good opportunities)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

data = pd.read_excel('/content/opportunities_with_opportunitylabel.xlsx')

# Encode the 'Country' column
le = LabelEncoder()
data['Country'] = le.fit_transform(data['Country'])
country_names = le.classes_  # Get the list of country names

# Define features
X = data[['Country', 'Total_Keyword_Weight']]

# Create a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the entire dataset
model.fit(X, data['OpportunityLabel'])

# Predict the probability of each country being in the "better" category
predicted_probabilities = model.predict_proba(X)[:, 1]

# Add the predicted probabilities to the original dataset
data['Probability_Better_Opportunity'] = predicted_probabilities

# Decode the country labels back to their original names
data['Predicted_Country'] = le.inverse_transform(data['Country'])

# Group the data by country and take the maximum probability
grouped_data = data.groupby('Predicted_Country')['Probability_Better_Opportunity'].max().reset_index()

# Sort the countries by their predicted probabilities in descending order
sorted_countries = grouped_data.sort_values(by='Probability_Better_Opportunity', ascending=False)

# Display the sorted list of countries with their predicted probabilities
print(sorted_countries)

sorted_countries.to_excel('/content/sorted_countries_with_probabilities.xlsx', index=False)


   Predicted_Country  Probability_Better_Opportunity
6            Gauteng                            1.00
2       Eastern Cape                            0.92
11        Mpumalanga                            0.84
8      KwaZulu-Natal                            0.84
0             Angola                            0.00
16            Uganda                            0.00
15      South Africa                            0.00
14     Northern Cape                            0.00
13        North West                            0.00
12          National                            0.00
9            Limpopo                            0.00
10           Morocco                            0.00
1           Cameroon                            0.00
7              Ghana                            0.00
5             Gambia                            0.00
4         Free State                            0.00
3             France                            0.00
17      Western Cape                          

Regression models to predict the frequency of opportunities (I can not do this because I do not have the posting date.)

Indicators



*   Number of future tenders per country per sector
*   How many tenders we will have based on our target class
*  Which countries are in each target class







In [ ]:
df = pd.read_excel('/content/opportunities.xlsx')

# Group the data by 'Country' and 'Sector' and count the number of tenders in each group
tenders_per_country_per_sector = df.groupby(['Country', 'Sector']).size().reset_index(name='TenderCount')

# Count the occurrences of each class
tender_counts_by_target_class = df['Target Class'].value_counts().reset_index()
tender_counts_by_target_class.columns = ['Target Class', 'Count']


# Group the data by 'TargetClass' and aggregate the unique countries associated with each class
countries_by_target_class = df.groupby('Target Class')['Country'].unique().reset_index()

# Create a new DataFrame with 'Country' and 'Target Class' columns
result_df = countries_by_target_class.explode('Country')


result_df.columns = ['Target Class', 'Country']


with pd.ExcelWriter('/content/output_indicators.xlsx') as writer:
    tenders_per_country_per_sector.to_excel(writer, sheet_name='Tenders by Country & Sector', index=False)
    tender_counts_by_target_class.to_excel(writer, sheet_name='Target Class Counts', index=False)
    countries_by_target_class.apply(pd.Series).to_excel(writer, sheet_name='Countries by Target Class', index=False)


Number of opportunities by year, by month and by day

In [ ]:
data = pd.read_excel('/content/opportunities.xlsx')

# "Deadline" column is in datetime format
data['Deadline'] = pd.to_datetime(data['Deadline'])

# Extract the year, month, and day from the "Deadline" column
data['Year'] = data['Deadline'].dt.year
data['Month'] = data['Deadline'].dt.month
data['Day'] = data['Deadline'].dt.day

# the total count of opportunities for every year
opportunities_by_year = data.groupby(['Year']).size().reset_index(name='Opportunity Count (Year)')

# the total count of opportunities for every month
opportunities_by_month = data.groupby(['Year', 'Month']).size().reset_index(name='Opportunity Count (Month)')

# the total count of opportunities for every day
opportunities_by_day = data.groupby(['Year', 'Month', 'Day']).size().reset_index(name='Opportunity Count (Day)')

# the overall total count of opportunities
total_opportunities = len(data)


print("Opportunities by Year:")
print(opportunities_by_year)
print("\nOpportunities by Month:")
print(opportunities_by_month)
print("\nOpportunities by Day:")
print(opportunities_by_day)


with pd.ExcelWriter('opportunity_counts.xlsx') as writer:
    opportunities_by_year.to_excel(writer, sheet_name='Opportunities by Year', index=False)
    opportunities_by_month.to_excel(writer, sheet_name='Opportunities by Month', index=False)
    opportunities_by_day.to_excel(writer, sheet_name='Opportunities by Day', index=False)




Opportunities by Year:
   Year  Opportunity Count (Year)
0  2023                       188
1  2024                        12
2  2025                         5
3  2026                         3
4  2027                         2
5  2030                         2

Opportunities by Month:
    Year  Month  Opportunity Count (Month)
0   2023      3                          1
1   2023      9                        123
2   2023     10                         53
3   2023     11                          2
4   2023     12                          9
5   2024      2                          1
6   2024      3                          1
7   2024      6                          2
8   2024      8                          1
9   2024      9                          1
10  2024     12                          6
11  2025      3                          1
12  2025     10                          3
13  2025     12                          1
14  2026      6                          1
15  2026     12           

Number of opportunities by year for every country

In [ ]:
data = pd.read_excel('/content/opportunities.xlsx')

# "Deadline" column is in datetime format
data['Deadline'] = pd.to_datetime(data['Deadline'])

# Filter the dataset to include only future tenders (those with a Deadline greater than or equal to today)
today = pd.to_datetime('today')
future_tenders = data[data['Deadline'] >= today]

# Extract the year
future_tenders['Year'] = future_tenders['Deadline'].dt.year

# Group the data by country, year, and count the number of future tenders for each group
future_tenders_by_country_year = future_tenders.groupby(['Country', 'Year']).size().reset_index(name='Tender Count')

print(future_tenders_by_country_year)

future_tenders_by_country_year.to_excel('future_tenders_by_country_year.xlsx', index=False)


         Country  Year  Tender Count
0         Angola  2023             2
1         Angola  2024             1
2         France  2023             6
3         France  2024             1
4         France  2026             1
5         Gambia  2023             1
6         Gambia  2025             1
7        Gauteng  2023             2
8          Ghana  2023             1
9          Ghana  2024             6
10         Ghana  2026             1
11       Morocco  2023             2
12       Morocco  2024             1
13       Morocco  2025             1
14       Morocco  2027             1
15       Morocco  2030             2
16  South Africa  2023             1
17  South Africa  2024             3
18  South Africa  2025             3
19        Uganda  2023             4
20        Uganda  2026             1
21        Uganda  2027             1
22  Western Cape  2023             2


<ipython-input-39-3d10b23489e7>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_tenders['Year'] = future_tenders['Deadline'].dt.year


All the opportunities based on country and month

In [ ]:
df = pd.read_excel('/content/opportunities.xlsx')

# "Deadline" column is in datetime format
df['Deadline'] = pd.to_datetime(df['Deadline'])

# Extract the month from the "Deadline" column
df['Month'] = df['Deadline'].dt.month

# Display the DataFrame with the "Country" and "Month" columns
result_df = df[['Country', 'Month']]
print(result_df)


with pd.ExcelWriter('/content/country_month_data.xlsx') as writer:
    df[['Country', 'Month']].to_excel(writer, sheet_name='Country vs Month', index=False)


           Country  Month
0          Gauteng      9
1       Mpumalanga      9
2          Gauteng      9
3     Eastern Cape     10
4          Gauteng      9
..             ...    ...
207  KwaZulu-Natal      9
208        Gauteng     10
209     Mpumalanga      9
210        Gauteng      9
211   Eastern Cape      9

[212 rows x 2 columns]


In [ ]:
!pip install xlsxwriter

In [ ]:
import os

excel_files = ['country_month_data.xlsx', 'feature_importance.xlsx', 'future_tenders_by_country_year.xlsx', 'opportunity_counts.xlsx', 'output_indicators.xlsx', 'predicted_opportunities.xlsx', 'sorted_countries_deadlines.xlsx', 'sorted_countries_with_probabilities.xlsx']

excel_writer = pd.ExcelWriter('predictions.xlsx', engine='xlsxwriter')

# giving a name to every sheet
for excel_file in excel_files:
    sheet_name = os.path.splitext(os.path.basename(excel_file))[0][:31]
    df = pd.read_excel(excel_file)
    df.to_excel(excel_writer, sheet_name=sheet_name, index=False)

excel_writer.save()


<ipython-input-42-3dbac25c7309>:13: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()
